In [1]:
# Declaring Dependencies
import os
import csv

import xlrd
import numpy as np
import pandas as pd

In [2]:
# Give the location of the file
loc = ("NACA Profiles.xlsx")

# Open the workbook
wb = xlrd.open_workbook(loc)

# Counting the number of sheets
res = len(wb.sheets())

# Saving the name of the excel file tabs
tabs = wb.sheet_names()
tabs = tabs[1:]

# Creating the folders, if needed.
current_dir = os.getcwd()
path1 = os.path.join(current_dir, "csvs")
path2 = os.path.join(current_dir, "txts")

if not os.path.isdir(path1):
    os.mkdir(path1)
if not os.path.isdir(path2):
    os.mkdir(path2)

In [3]:
# This is the reader for the sheets. All sheets but the 1st.
for sh in range(res-1):
    xc = []
    yc = []
    sh2 = sh+1
    c_sheet = wb.sheet_by_index(sh2)
        
    # This is the reader for the rows
    # This is keeping the first row
    for i in range(c_sheet.nrows):
        xc.append(c_sheet.cell_value(i, 0))
        yc.append(c_sheet.cell_value(i, 1))
        
    # This is where the main DataFrame is created   
    df = pd.DataFrame(list(zip(xc,yc)))
      
    # Take the data minus the header row
    df = df[1:]
        
    # Calculating the coordinates of a final point based on the strongest slope
    
    # Getting the coordinates of the first 2 points
    x1 = df.iloc[0,0]
    y1 = df.iloc[0,1]
    x2 = df.iloc[1,0]
    y2 = df.iloc[1,1]

    # Getting the coordinates of the last 2 points
    xn = df.iloc[-1,0]
    yn = df.iloc[-1,1]
    xn_1 = df.iloc[-2,0]
    yn_1 = df.iloc[-2,1]
    
    # Calculating the slopes and comparing them
    # The criteria used here will be to use the more abrupt closing shape,i.e.,the highest slope.

    # Linear approximation calculation
    mu = (y1-y2)/(x1-x2)
    ml = (yn-yn_1)/(xn-xn_1)
    nu = y1-mu*x1
    nl = yn-ml*xn
    
    # Final coordinates of new point
    x3 = (nl-nu)/(mu-ml)
    y3 = nu+mu*x3
    
    # Appending the last value to the list
    df_list = df.values.tolist()
    new_point = [x3,y3]
    df_list.append(new_point)
     
    # Droping duplicates   
    drop_dups = pd.Series(df_list).drop_duplicates().tolist()
    
    # Setting the header of the DataFrame
    new_header = [['x_coord', 'y_coord']]
     
    # Recreating the DataFrame with last point
    df = pd.DataFrame(drop_dups,columns=new_header)
    
    # DataFrames edited to match Fluent format
    df["#Group"] = 1
    df['#Point'] = df.index+1
    df["z_coord"] = 0
    
    # Reordering the columns to match Fluent requirement
    df = df[['#Group', '#Point', 'x_coord', 'y_coord', 'z_coord']]
    
    # Creating a last line to match format
    df3 = pd.DataFrame([[1,0,np.nan,np.nan,np.nan]],columns=[['#Group','#Point','x_coord','y_coord','z_coord']])
    
    # Appending last line to original DataFrame
    df = df.append(df3, ignore_index=True)
   
    # DataFrames saved to csv files
    df.to_csv(path1 + r'\perfil_{0}.csv'.format(tabs[sh]), index=False)

    # Creating and saving the text files from the csv files
    with open(path2 + '\\txt_file_{0}.txt'.format(tabs[sh]), "w") as my_output_file:
        with open(path1 + '\perfil_{0}.csv'.format(tabs[sh]), "r") as my_input_file:
            [ my_output_file.write("\t".join(row)+'\n') for row in csv.reader(my_input_file)]
        my_output_file.close()